# Notes for Computational Fluid Dynamics

The Basics with Applications by John D. Anderson Jr.

In [3]:
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt 

## Pre-Thoughts

### Principle components to balance
1. Mass flow in/out of a unit
2. Pressure exerted by surrounding units
3. Fluid velocity
4. 2 dimensional effects (flow in y affecting x)
5. Boundary conditions for the entire model
6. Defining the shape of an object to test
7. Flow rotation?

### Interesting Thoughts
1. Data structure for efficiently sub-dividing the space where needed
2. Iterative approximations of the solution
3. Parallel evaluation
4. Defining the shape of an object not as a cube so that I don't have to infinitely subdivide a cube/square
5. How can I determine boundary conditions to evaluate when a unit has subdivided units on 1 or more sides?
6. How can I determine when two units can be recombined into a larger evaluation unit to save future computation?

### What are my inputs?
- Defining a hull shape by blocks or otherwise would be "faster"
- Defining a hull shape by mathematical functions might not work as well with a grid, but would allow for optimizations as a derivative of different parameters.

### What are my outputs?
- A heatmap of pressure?
- Drag calculations?
    - "parasitic drag": form drag, skin friction
    - lift induced drag (planing hulls)
    - wave drag/resistance at a fluid boundary [Wikipedia](https://en.wikipedia.org/wiki/Wave-making_resistance)
        - "For small displacement hulls, such as sailboats or rowboats, wave-making resistance is the major source of the marine vessel drag."
        - I think this is a function of pressure build up, but I'd have to read more
- Velocity vector visualization
- Approximate particle paths. Probably more of an interesting visual than a useful tool, but idk. If I'm building the full pressure/velocity map, why not?
- Visualization of the mesh used for the calculation

## Other References
- [Sail Performance](https://www.amazon.com/Sail-Performance-Techniques-Maximize-Power/dp/0070402507)

## Road Map
1. Fluid Flow Equations
2. Mathematical Behavior of the Equations
3. Basic discretization of the Equations
4. Finite difference, Finite Volume
5. Transforms and Grid Generation
6. Popular Techniques
7. Detailed Worked Examples
    - Viscous flow x2

# Chapter 2

## 2.1 Basics
1. Mass is conserved
2. Newton's second law, F = ma
3. Energy is conserved

## 2.2 Models of Flow

### 2.2.1 Finite Control Volume

"Large", finite volume containing many particles either fixed in space with fluid moving through it or fixed to the fluid with the same particles contained in it.

Apply physics constraints to the fluid contained in the volume and to fluid crossing the surface. This allows one to limit the consideration to just the control volume as opposed to the entire fluid. Fluid equations are "direct" and "integral" form. The integral forms are manipulated to *indirectly* get to partial differential equations. The control volume fixed in space are the *conservation* form.

### 2.2.2 Infinitesimal Fluid Element

"Small" volume (cube?) with fluid passing through it or fixed to fluid and moving along a given streamline

This leads directly to the partial differential equations formulation.

## 2.3 Substantial Derivative

The "substantial" derivative of density for a small volume following a velocity flow between two fixed times is:

For this section, p is rho, is density instead of pressure

Dp / Dt = (dp/dx) * (x2 - x1)/(t2 - t1) + (dp/dy) * (y2 - y1)/(t2 - t1) + (dp/dz) * (z2 - z1)/(t2 - t1) + (dp/dt)

In the limit, (x2 - x1)/(t2 - t1) etc. are velocities in u, v, w (x, y, z respectively)

Dp / Dt = u * (dp/dx) + v * (dp/dy) + w * (dp/dz) + (dp/dt)

dp / dt is the derivative of density at a fixed point w.r.t. time

This notation can be applied to any fluid property, for example: density, static pressure, static? temperature, velocities in a particular direction (I think)

## 2.4 Divergence of Velocity

Physical meaning: The time rate of change of the volume of a moving fluid element per unit volume, derived for a small uniform volume containing a fixed number of fluid particles. Math written as 

(upside down Delta) dot V = (1 / dV) (D(dV) / Dt)

Where (upside down Delta) is the vector operator, V is the volume, dV is the infinitesmal volume, D/Dt is the substantial derivative

## 2.5 Continuity Equation

Applied to the physical principle: Mass is conserved.

Net Mass Flow out of control volume = B

Time Rate of decrease of mass in control volume = C

$$B = C$$

### 2.5.1 Derivation for Fixed Finite Control Volume

aka "large" control volume not moving with flow, conservative integral form

$\mathbf{V}$ = flow velocity, $\mathbf{dS}$ = vector elemental surface area, $d\mathscr{V}$ = elemental volume inside control volume

$$B = \iint_{S} \rho \mathbf{V} \cdot \mathbf{dS}$$

$$- \frac{\partial}{\partial t} \iiint_{\mathscr{V}} \rho d \mathscr{V} = C$$

Combining the two:

$$- \frac{\partial}{\partial t} \iiint_{\mathscr{V}} \rho d \mathscr{V} = C = B = \iint_{S} \rho \mathbf{V} \cdot \mathbf{dS}$$

$$\frac{\partial}{\partial t} \iiint_{\mathscr{V}} \rho d \mathscr{V} + \iint_{S} \rho \mathbf{V} \cdot \mathbf{dS} = 0$$

Derivative of volume times density w.r.t. time + mass moving out through volume surface = 0

aka conservation of mass 🙂

### 2.5.5 They're all the same!

It talks through different methods, but I like 2.5.1 for now

### 2.5.6 Mathematical note

The PDE models assume that the flow is differentiable, while the integral ones make no assumptions about flow within the volumes, making the integral one more "fundamental" and better able to handle real discontinuities, such as shock waves.

In [8]:
%%latex
$$A_{s} = B_{z}$$

<IPython.core.display.Latex object>

## Chapter 2 Thoughts

I'm going to make assumptions:
- Constant density (incompressible fluid?)
- Constant temperature

In [9]:
print('Hello World')

Hello World


## 2.6 Momentum Equation

In the text, this is derived for the moving fluid element model. I'm also going to derive it for the fixed control volume model.

$$\mathbf{F} = m \mathbf{a}$$

This is a vector relation, so can be considered along each axis independently.

2 types of forces:
1. Body force (gravity, electric, magnetic)
2. Surface forces
    - Pressure distribution from surrounding fluid acting on the surface (perpendicular to surface)
    - Viscous Forces: Shear and normal stress distributions acting on the surface imposed by the outside fluid tugging or pushing by friction
    
Body force on fluid element in x direction = $\rho f_{x}(dx, dy, dz)$

Where $\rho$ is density, $f_{x}$ is force per unit mass, and $(dx, dy, dz)$ is the volume of the small mass considered in the derivation.

The shear and normal stresses are related to the time rate of change of the deformation of the fluid element. Shear stress along a surface warps the small volume into a trapezoid, normal stress leads to elongation (in the positive direction) of that small volume. Both depend on velocity gradients in the flow. In most viscous flows, normal stress is smaller than shear stress and is neglected. Normal stress is very important when the normal velocity gradient is large (e.g. inside a shock wave).

Convention: $\tau_{ij}$ = stress in the $j$ direction acting on a surface perpendicular to the $i$ direction.

Note to self: This will get interesting if/when I'm using hyperplanes that aren't cartesian.

Combining forces:

$$\mathbf{F}_{x} = [-\frac{\partial p}{\partial x} + \frac{\partial \tau_{xx}}{\partial x} + \frac{\partial \tau_{yx}}{\partial y} + \frac{\partial \tau_{zx}}{\partial z}](dx \cdot dy \cdot dz) + \rho f_{x}(dx \cdot dy \cdot dz)$$

$F_{x}$ = \[pressure force + normal stress + y-shear stress + z-shear stress\](across the small volume) + body forces = left hand side of F=ma

$m$ = $\rho (dx \cdot dy \cdot dz)$

$\mathbf{a}$ = change in x velocity $u$

For a moving small body, use the substantial derivative here, so

$$\mathbf{a} = \frac{D u}{D t}$$

Bringing everything back together:

$$-\frac{\partial p}{\partial x} + \frac{\partial \tau_{xx}}{\partial x} + \frac{\partial \tau_{yx}}{\partial y} + \frac{\partial \tau_{zx}}{\partial z} + \rho f_{x} = \rho \frac{D u}{D t}$$

Which is the x component of the momentum equation for viscous flow on a moving small element. Y and Z versions:

- $$-\frac{\partial p}{\partial y} + \frac{\partial \tau_{xy}}{\partial x} + \frac{\partial \tau_{yy}}{\partial y} + \frac{\partial \tau_{zy}}{\partial z} + \rho f_{y} = \rho \frac{D v}{D t}$$
- $$-\frac{\partial p}{\partial z} + \frac{\partial \tau_{xz}}{\partial x} + \frac{\partial \tau_{yz}}{\partial y} + \frac{\partial \tau_{zz}}{\partial z} + \rho f_{z} = \rho \frac{D w}{D t}$$

With some manipulation, we get the canonical form of Navier Stokes equations.

### 2.6 My Derivation

aka the fixed control volume model.

$$\mathbf{F} = m \mathbf{a}$$

This is a vector relation, so can be considered along each axis independently (starting with $x$)

$$F_{x} = m a_{x}$$

#### Body Forces

$$\iiint_{\mathscr{V}} \rho f_{x} d \mathscr{V}$$

(assuming that the only body force is gravity, so the unit body force $f_{x}$ is constant across the volume)

$$\rho f_{x} \mathscr{V}$$

#### Surface Forces

For some small vector surface $d \mathbf{S}$, there are forces parallel to $d \mathbf{S}$ and perpendicular to $d \mathbf{S}$. For normal forces (parallel to surface vector) caused by velocity gradients $\tau_{ss}$, their x component is: 

$$\tau_{sx} = \tau_{ss} (d \mathbf{S} \cdot i)$$ where $i$ is the unit vector parallel to the x axis.

For the general case of shear forces, there is a set of vectors perpendicular to $d \mathbf{S}$, so for all $\tau_{s \bot}$ perpendicular to $d \mathbf{S}$, the x component is:

$$\tau_{sx} = \sum_{\bot} \tau_{s \bot} (\vec{\bot} \cdot i)$$ where $i$ is the unit vector parallel to the x axis. This can probably be simplified down to considering the summed x, y and z components of the shear forces, which is implicitly jumping to $\tau_{sx}$.

#### Pressure Forces

For some pressure gradient $\delta p$ along $d \mathbf{S}$, the force is:

$$- \iint_{S} \frac{\delta p}{\delta S} d \mathbf{S}$$

For the x component:

$$- \iint_{S} \frac{\delta p}{\delta x} (d \mathbf{S} \cdot i)$$

Assuming the pressure is approximately constant across the surface, (e.g. not a large change in height or otherwise) the pressure force in x from each surface is:

$$- \frac{\delta p}{\delta x} (\vec{S} \cdot i)$$

#### Mass

$$\iiint_{\mathscr{V}} \rho \delta V$$

#### Acceleration

For an internal flow velocity $u$ in the x direction (note lack of substantial derivative):

$$a_{x} = \frac{\delta u}{\delta t}$$

## 2.7 The Energy Equation

### My worked version

Rate of change of energy inside fluid element = (net flux of heat in + rate of work done on element due to body and surface forces)

$$A = (B + C)$$

Note to self: If the element isn't moving, the work done on the element is 0, and I'm assuming no heat flux because no delta in temperature, so the rate of change of the energy inside the fluid element is 0.

For component C, 2 elements:
Work done by body forces: $\rho f_{x} \cdot \mathbf{V}(dx, dy, dz)$

With the fluid element moving at a velocity $\mathbf{V}$

The other component is work done by shear/normal forces which is big and messy and every term has a component of velocity of the control volume, which I'm fixing to 0, so it goes to zero as well.

B = 0 because I'm assuming no heat transfer

The A has two components:
1. Random molecular motion (rotation, translation, vibration) $e$ per unit mass
2. Kinetic energy due to translational movement of the fluid element $\frac{V^{2}}{2}$ per unit mass

For a fixed control volume, the time rate of change of the total energy per unit mass is the usual derivative (not substantial). The mass is $\iiint_{\mathscr{V}} \rho \delta \mathscr{V}$ we get:

$$A = \frac{\delta}{\delta t} \left( e + \frac{V^{2}}{2} \right) \iiint_{\mathscr{V}} \rho \delta \mathscr{V} $$

If I make the simplifying assumption that the internal energy $e$ doesn't change significantly over time ($\frac{\delta e}{\delta t} = 0$):

$$A = \frac{\delta}{\delta t} \left(\frac{V^{2}}{2} \right) \iiint_{\mathscr{V}} \rho \delta \mathscr{V} $$

Which would seem to suggest that $\frac{\delta}{\delta t} \frac{V^{2}}{2} = 0$ and $\frac{\delta}{\delta t} V^{2} = 0$ and $\frac{\delta}{\delta t} V = 0$. Except that there is some fluid flow in and out, so that the energy changes due to some fluid flowing in with energy per unit mass $\frac{V_{in}^{2}}{2}$ for the mass influx and $\frac{V_{out}^{2}}{2}$ for the mass outbound. There is no work done on the cell, but forces move mass around?

## 2.8 Summary of Equations

### 2.8.2 Euler Equations
aka drop the friction and thermal conduction terms :)

#### Continuity Equation

Conservative form:

$$\frac{\partial p}{\partial t} + \nabla \cdot (\rho \mathbf{V}) = 0$$

#### Momentum Equation

Conservative form, x component:

$$\frac{\partial (\rho u)}{\partial t} + \nabla \cdot (\rho u \mathbf{V}) = -\frac{\partial p}{\partial x} + \rho f_{x}$$

#### Energy Equation

Conservation form:

$$\frac{\partial }{\partial t} \left[ \rho \left( e + \frac{V^{2}}{2} \right) \right] + \nabla \cdot \left[ \rho \left( e + \frac{V^{2}}{2} \right) \mathbf{V} \right] =$$

$$ \rho \dot{q} - \frac{\partial (up)}{\partial x} - \frac{\partial (vp)}{\partial y} - \frac{\partial (wp)}{\partial z} + \rho f \cdot \mathbf{V}$$

## 2.9 Boundary Conditions

For viscous flow, there is a no-slip condition at the surface:

$$u = v = w = 0$$

For inviscid flow, the "no slip" condition is replaced by non-tangent flow, aka the fluid can't flow into or out of the wall, and can only flow tangentially.

For a body in a free stream, the boundary conditions sufficiently far away from the object are the velocity/pressure/etc. of the freestream.

## 2.10 Considerations for CFD

Conservative forms can be written as matrix math. This math gives the derivative for the particular components at that point in time, which can be used to step through time to either get unsteady solution information or to get a steady state solution by "marching" through time until the solution settles down.

Inviscid flow has simpler column vectors.

Marching solutions can also march through space, but I'm interested in time.

Shock waves can be either "captured" and allowed to happen accidentally, with the potential for "low" resolution of a handful of grid areas or "fitted" where a mathematical shape for a shockwave is placed and then explicit transitions from pre-shock to post shock conditions are solved at that point. These two solutions can be combined, but I don't anticipate having any, so I'm going to stick to a "capture" approach. For a shock capturing method, a conservative approach should be used.

This does impress upon me that, to increase the resolution of potential shock waves or similar effects, the difference in boundary conditions indicating that a shock occured can be subdivided parallel to the shock effect to get a greater resolution for its exact position in space.

I like my conservative methods :)

Note to self: Stopping here for now, resume on page 76 of the pdf/pg. 125 of the text